In [28]:
import requests
from PIL import Image, ImageDraw
from io import BytesIO
import gradio as gr


IMAGE_URL = "https://cdn.pixabay.com/photo/2019/08/25/13/34/dogs-4429513_1280.jpg"
BASE_ENDPOINT = "https://fimtrus-vision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze"
API_KEY="1Xvo96LQT7LFAe6FgQUeO3f8UwOfELLcvZNJThBQW6LJsBdUhwzrJQQJ99BGACYeBjFXJ3w3AAAFACOGxEEM"


def random_color():
    import random
    # 랜덤한 RGB 색상 튜플 반환
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 


def get_font():
    # OS별로 적절한 폰트 객체 반환 (한글 지원)
    from PIL import ImageFont
    import platform
    
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux      
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 폰트 파일이 없을 경우 기본 폰트 사용
        return ImageFont.load_default(size=font_size)
    
# features에 따라 response가 변함
# features가 caption일 때 gender-neutral-caption을 추가해줘야 한다. 아예 빠지던지, 추가되던지 해야함.
# features smartCorps 일 때, smart-aspect-ratios 를 추가해야한다. 단, 빈값도 가능.

def request_image_analysis(image_url,features=['objects'],**kwargs):
    endpoint="{}".format(BASE_ENDPOINT)
    params={
        "api-version":"2024-02-01",
        "features":",".join(features)
    }
    headers={
        "Ocp-Apim-Subscription-Key":API_KEY
    }
    body={
        "url":image_url   
    }

    if kwargs is not None:
        if ("caption" in features or "denseCaptions" in features) and "gender_neutral_caption" in kwargs.keys():
            params.update({
                "gender-neutral-caption":kwargs['gender_neutral_caption']
            })
        if "smartCrops" in features and "smartcrops_aspect_ratios" in kwargs.keys():
            params.update({
                "smartcrops-aspect-ratios":kwargs['smartcrops_aspect_ratios']
            })

        print(params)

        response=requests.post(endpoint,params=params,headers=headers,json=body)

        if response.status_code !=200:
            return None
        
        response_json=response.json()
        return response_json



def draw_image(image_url, features, data):
    # 이미지 다운로드
    image_response = requests.get(image_url)
    image = Image.open(BytesIO(image_response.content))
    draw = ImageDraw.Draw(image)
    font = get_font()

    for feature in features:
        if feature == "tags" or feature == "caption":
            continue
        feature_key = '{}Result'.format(feature)
        print(feature_key)
        # 객체인식 결과 리스트
        result_object = data[feature_key]
        block_list = result_object['values']

        color = random_color()

        for block in block_list:

            bounding_box = block['boundingBox']
            x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']

            if feature == "objects":
                tag = block['tags'][0]       
                name = tag['name']
                confidence = tag['confidence']
                
                formatted_text = "{} ({:.2f}%)".format(name, confidence * 100)
            
            elif feature == "denseCaptions":
                formatted_text = block["text"]
            else:
                formatted_text=None
            
            # 객체 위치에 사각형과 라벨 그리기
            draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=2)

            # Feature의 텍스트를 그려주는 부분
            feature_text_bbox=draw.textbbox((x, y),feature, font=font)
            draw.rectangle([(feature_text_bbox[0],feature_text_bbox[1]),(feature_text_bbox[2],feature_text_bbox[3])],outline=color,fill=color,width=2)
            draw.text((x,y),feature,fill="black",font=font)

            # 텍스트를 그려주는 부분
            if formatted_text:
                text_bbox=draw.textbbox((x,y+20),formatted_text,font=font)
                draw.rectangle([(text_bbox[0],text_bbox[1]),(text_bbox[2]+5,text_bbox[3])],outline=color,fill=color,width=2)
                draw.text((x+5,y+20),formatted_text,fill="white",font=font)
        
    return image

def get_result_text(features,response_data):
    result_text_list=[]
    # 리스트에 담는 이유는 join으로 한번에 출력하기 위함이다.

    if "caption" in features:
        result_text_list.append("[Caption]")
        result_text_list.append(response_data['captionResult']['text'])
        result_text_list.append("{:.2f}%".format(response_data['captionResult']['confidence']*100))
    
    if "tags" in features:
        if len(result_text_list) > 0:
            result_text_list.append("\n")

        result_text_list.append("[Tags]")
        tag_list = response_data['tagsResult']['values']
        for tag in tag_list:
            name= tag['name']
            confidence=tag['confidence']
            result_text_list.append("{}({:.2f}%)".format(name, confidence*100))
    result_text="\n".join(result_text_list)

    return result_text




theme = gr.themes.Soft(primary_hue=gr.themes.colors.indigo)
with gr.Blocks(theme=theme) as demo:

    # feature 정의: 다중선택이 가능하도록 함수 밖으로 
    features=["objects","caption","denseCaptions","tags","smartCrops"]
    
    # click_send : 버튼을 클릭했을 때, 어떤 동작이 일어아냐 하나요?
        # url이 request_image_analysis로 전달되고, features의 값도 전달이 함께 되어야 한다. + is_gender_neutural, ratio
    def click_send(image_url,features,is_gender_neutral,ratio):

        #features는 dic형태로 받아야하고, option 내에 caption이나, smartCrops가 포함된다면 그 값으로 변경 적용된다. (bool)
        option={}

        if "caption" in features or "denseCaptions" in features:
            option.update({'gender_neutral_caption':is_gender_neutral})
        if "smartCrops" in features:
            option.update({'smartcrops_aspect_ratios':ratio})

        response_data=request_image_analysis(image_url,features,**option)
        image=draw_image(image_url,features,response_data)
        result_text=get_result_text(features, response_data)

        print(response_data['captionResult'])
        print(response_data['tagsResult'])
        # To Do: response_data에서 받은 caption, tags정보를 가공해서, text로 만든 뒤, return 해준다.
        return image, result_text


    # feature를 바꿔서 선택할 수 있는 함수를 만들어야 함.
    def change_features(features):
        print(features)
        selected_caption=False
        selected_smart_crops=False

        if "caption" in features or "denseCaptions" in features:
            selected_caption=True
        if "smartCrops" in features:
            selected_smart_crops = True

        return features, gr.update(visible=selected_caption,interactive=True),gr.update(visible=selected_smart_crops,interactive=True)    
    
    # # change : gender_neutral_caption
    # def change_gender(is_neutral):
    #     print(is_neutral)
    #     return is_neutral
    # # input : smartcrops_aspect_ratios

    # UI
    features_checkbox=gr.CheckboxGroup(label="Features",choices=features)
    # gender_neutral_caption
    gender_radio=gr.Radio(label="성중립성",choices=[("중립",True),("구분",False)],value=False,visible=False)
    # smartcrops_aspect_ratios
    ratio_text_box=gr.Textbox(label="smartCrops 크기",placeholder="ex. 0.75,1.2,1.5",visible=False)
    image_url_textbox=gr.Textbox(label="이미지 URL", placeholder="여기에 이미지 url을 붙여 넣으세요.")
    send_button=gr.Button("전송")    

    with gr.Row():
        output_image=gr.Image(scale=3,label="결과확인", interactive=False,type='pil')
        output_texbox=gr.TextArea(scale=1,label="결과텍스트",lines=20)

    # trigger (send_buttuon, feaures_checkbox, gender_radio)

    send_button.click(click_send,inputs=[image_url_textbox,features_checkbox,gender_radio,ratio_text_box],outputs=[output_image,output_texbox])
    features_checkbox.change(change_features,inputs=[features_checkbox],outputs=[features_checkbox,gender_radio,ratio_text_box])
demo.launch()

# # 객체 인식 결과 요청 및 시각화
# features=["objects","caption","denseCaptions","tags","smartCrops"]
# response_json2 = request_image_analysis(IMAGE_URL,features,gender_neutral_caption=False,smartcrops_aspect_ratios="0.7,1.8")
# draw_image(IMAGE_URL,features,response_json2)


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


['tags']
['tags', 'denseCaptions']
['tags', 'denseCaptions', 'caption']
{'api-version': '2024-02-01', 'features': 'tags,denseCaptions,caption', 'gender-neutral-caption': False}
denseCaptionsResult
{'text': 'a woman in a white dress with a white horse', 'confidence': 0.8510650992393494}
{'values': [{'name': 'outdoor', 'confidence': 0.9910329580307007}, {'name': 'grass', 'confidence': 0.9866957664489746}, {'name': 'bride', 'confidence': 0.98617023229599}, {'name': 'wedding dress', 'confidence': 0.9824004173278809}, {'name': 'cloud', 'confidence': 0.9810846447944641}, {'name': 'sky', 'confidence': 0.9731866717338562}, {'name': 'dress', 'confidence': 0.9538217782974243}, {'name': 'person', 'confidence': 0.9229608774185181}, {'name': 'gown', 'confidence': 0.8891831636428833}, {'name': 'female person', 'confidence': 0.8782088756561279}, {'name': 'bridal clothing', 'confidence': 0.8504803776741028}, {'name': 'horse', 'confidence': 0.8493216037750244}, {'name': 'plant', 'confidence': 0.8417814